In [1984]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli, pauli_basis, SuperOp, PTM, Operator
from qiskit.circuit.library import CXGate, CZGate, HGate, SGate, SdgGate
from random import choice, choices
from itertools import product, permutations, cycle
from scipy.optimize import curve_fit, nnls
from matplotlib import pyplot as plt
import numpy as np
from qiskit.providers.aer.noise import NoiseModel

In [1927]:
#display a progress bar for my sanity
def progressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    if iteration == total: 
        print()

In [2168]:
#generate circuits and metadata for learning the noise in a layer of single-qubit gates
class NoiseLearningExperiment:
    def __init__(self, layer, backend, samples=64, depths = [2,4,8,16,32,64,128]):
        #initialize variables
        self.layer = layer
        self.depths = depths
        self.backend = backend
        self.samples = samples

        self.n = len(layer) #number of qubits nontrivially affected in layer
        self.num_qubits = len(backend.properties().to_dict()["qubits"])

        #parse coupling map to contain only qubits in layer
        proc_coupling = backend.configuration().coupling_map
        self.coupling_map = [(q1, q2) for q1,q2 in proc_coupling if q1 in layer and q2 in layer]

        #generate adjacency matrix, pauli strings, and transpilation dictionaries for later use
        self.adjacency_matrix = self.get_connectivity()
        self.pauli_strings = self.get_pauli_strings()
        self.raw_gates = {}
        self.transpiled_gates = {} #store dictionary of transpiled gates
        self.raw_gates, self.transpiled_gates = self._generate_gate_dict()
        self.conjugate_gates = self._generate_conjugate_gates()

    def unmapped(self, i):
        return self.layer.index(i)

    #parse coupling map to generate adjacency matrix and find the degree of each
    def get_connectivity(self):
        qubits = self.layer
        connections = self.coupling_map.copy() 
        n = self.n

        #remap qubits so that layer is sequential
        verts = [self.unmapped(qubit) for qubit in qubits]
        edges = [(self.unmapped(qubit1), self.unmapped(qubit2)) for qubit1,qubit2 in connections]

        #adjacency matrix has a 1 at i,j if i and j are connected, 0 otherwise
        adjacency_matrix = [[0 for i in verts] for j in verts] 
        for (vert1,vert2) in edges:
            adjacency_matrix[vert1][vert2] = 1
            adjacency_matrix[vert2][vert1] = 1

        return adjacency_matrix

    #The method acts as a head for the recursive sweeping procedure used to generate the pauli strings
    def get_pauli_strings(self):
        verts = range(self.n) #qubits in layer remapped in numerical order
        start_vertex = 0 #this should be chosen to be an edge qubit
        self.pauli_strings = [['I']*self.n for i in range(9)]
        visited_verts = [] #keep track of veritices for which the bases are already selected
        while(True): #if there are isolated regions _getstr bottoms out, this method restarts it
            remaining_verts = [v for v in verts if v not in visited_verts]
            if len(remaining_verts) == 0:
                #rearrange in numerical order
                self.pauli_strings = [Pauli("".join(string[::-1])) for string in self.pauli_strings]
                return self.pauli_strings 
            else: 
                self._getstr(remaining_verts[0], visited_verts)

    #recursive sweeping procedure to find pauli strings based on number of predecessor vertices
    def _getstr(self, vertex, visited_verts):
        pauli_strings = self.pauli_strings
        adjacency_matrix = self.adjacency_matrix
        
        #copied from Fig. S3 in van den Berg
        example_orderings = {"XXXYYYZZZ":"XYZXYZXYZ",
                            "XXXYYZZZY":"XYZXYZXYZ",
                            "XXYYYZZZX":"XYZXYZXYZ",
                            "XXZYYZXYZ":"XYZXZYZYX",
                            "XYZXYZXYZ":"XYZZXYYZX"}

        visited_verts.append(vertex)
        children = [i for i,e in enumerate(adjacency_matrix[vertex]) if e == 1]
        visited_children = [c for c in children if c in visited_verts]

        match len(visited_children):
            case 0:
                cycp = cycle("XYZ")
                for i,s in enumerate(pauli_strings):
                    pauli_strings[i][vertex] = next(cycp)

            case 1:
                predecessor = visited_children[0]
                #store permutation of indices so that predecessor has X,X,X,Y,Y,Y,Z,Z,Z
                reorder_list = [[] for i in range(3)]
                for i in range(9):
                    basis = pauli_strings[i][predecessor]
                    reorder_list["XYZ".index(basis)].append(i)
                
                for i in range(3):
                    for j,c in enumerate("XYZ"):
                        idx = reorder_list[i][j]
                        pauli_strings[idx][vertex] = c

            case 2:
                predecessor0 = visited_children[0]
                predecessor1 = visited_children[1]

                #use the same reordering trick to get XXXYYYZZZ on first predecessor
                reorder_list = [[] for i in range(3)] 
                for i in range(9):
                    basis = pauli_strings[i][predecessor0]
                    reorder_list["XYZ".index(basis)].append(i)
                
                #list out string with permuted values of predecessor 2
                substring = ""
                for list in reorder_list:
                    for idx in list:
                        substring += pauli_strings[idx][predecessor1]

                #match predecessor two with a permutation of example_orderings
                reordering = ""
                for perm in permutations("XYZ"):
                    p_string = "".join(["XYZ"[perm.index(p)] for p in substring])
                    if p_string in example_orderings:
                        reordering = example_orderings[p_string]
                        break
                
                #unpermute the example orderings so that they match the original strings
                i = 0
                for list in reorder_list:
                    for idx in list:
                        pauli_strings[idx][vertex] = reordering[i]
                        i += 1

            case _: #processor needs to have connectivity so that there are <= 2 predecessors
                raise Exception("Three or more predecessors encountered")
        
        for c in children: #call recursive method on children
            if c not in visited_children:
                self._getstr(c, visited_verts)

    def _generate_gate_dict(self):
        transpiled_gates = {}
        raw_gates = {}
        backend = self.backend

        for p in pauli_basis(1, pauli_list = True):
            qc = QuantumCircuit(1)
            qc.append(p, [0])
            transpiled_gates[p.to_label()] = transpile(qc, basis_gates = backend._basis_gates())
            raw_gates[p.to_label()] = qc

        for h,p in product(["H","HS", "SdgH", "S", "Sdg"],["I","X","Y","Z",""]):
            qc = QuantumCircuit(1)
            match h:
                case "H":
                    qc.h(0)
                case "HS":
                    qc.h(0)
                    qc.s(0)
                case "SdgH":
                    qc.sdg(0)
                    qc.h(0)
                case "S":
                    qc.s(0)
                case "Sdg":
                    qc.sdg(0)

            if p!="":
                qc.append(Pauli(p),[0])

            transpiled_gates[h+p] = transpile(qc, basis_gates = backend._basis_gates())
            raw_gates[h+p] = qc

        transpiled_gates['I'].id(0)
        transpiled_gates['Z'].id(0)


        return raw_gates, transpiled_gates

    def print_gate_conversions(self):
        raw_gates = self.raw_gates
        transpiled_gates = self.transpiled_gates

        qc = QuantumCircuit(len(raw_gates))
        for i,p in enumerate(raw_gates.keys()):
            qc = qc.compose(raw_gates[p], [i])


            qc2 = QuantumCircuit(len(transpiled_gates))

        for i,p in enumerate(transpiled_gates.keys()):
            qc2 = qc2.compose(transpiled_gates[p], [i])

        qc.barrier()
        qc = qc.compose(qc2)
        return qc

In [2782]:
from typing import NewType, Tuple

TwoQubitGate = NewType("TwoQubitGate", Tuple[str, Tuple])
#generate circuits and metadata for learning the noise in a layer of single-qubit gates
class TwoQubitGateLayerExperiment(NoiseLearningExperiment):

    '''
    layer: a list of qubit with gates contained in the layer being benchmarked
    backend: IBMQBackend to be run on 
    samples: samples from twirl to take
    depths: noise repetitions to use for exponential fit
    '''
    def __init__(self, layer, backend, two_qubit_gates, context_qubits=[], samples=64, depths = [2,4,8,16,32,64,128]):
        super().__init__(layer, backend, samples, depths)
        self.two_qubit_gates = two_qubit_gates
        self.context_qubits = context_qubits
        self.conjugate_gates = self._generate_conjugate_gates()
        self.pauli_sets = self.get_pauli_sets()

    def _generate_conjugate_gates(self):
        pauli_group = pauli_basis(2, pauli_list = True)
        pauli_dict = {p:p.to_matrix() for p in pauli_group}

        def without_phase(pauli):
            return Pauli((pauli.z, pauli.x))

        def conjugate(p, c):
            return without_phase(Pauli(get_name(c @ p @ np.conjugate(np.transpose(c)))))

        def get_name(pauli):
            for p in pauli_group:
                if Operator(pauli_dict[p]).equiv(Operator(pauli)):
                    return p

        gates_dict = {'cx':{}, 'cz':{}, 'H':{}, 'S':{}, 'Sdg':{}, 'SdgH':{}, 'HS':{}}
        c = CXGate().to_matrix()
        for p in pauli_group:
            gates_dict['cx'][p] = conjugate(pauli_dict[p],c)

        c = CZGate().to_matrix()
        for p in pauli_group:
            gates_dict['cz'][p] = conjugate(pauli_dict[p],c)

        pauli_group = pauli_basis(1, pauli_list = True)
        pauli_dict = {p:p.to_matrix() for p in pauli_group}

        c = HGate().to_matrix()
        for p in pauli_group:
            gates_dict['H'][p] = conjugate(pauli_dict[p],c)

        c = SGate().to_matrix()
        for p in pauli_group:
            gates_dict['S'][p] = conjugate(pauli_dict[p],c)

        c = SdgGate().to_matrix()
        for p in pauli_group:
            gates_dict['Sdg'][p] = conjugate(pauli_dict[p],c)

        c = HGate().to_matrix() @ SdgGate().to_matrix()
        for p in pauli_group:
            gates_dict['SdgH'][p] = conjugate(pauli_dict[p],c)

        c = SGate().to_matrix() @ HGate().to_matrix()
        for p in pauli_group:
            gates_dict['HS'][p] = conjugate(pauli_dict[p],c)

        return gates_dict

    def get_conjugate(self, gate, pauli):
        return self.conjugate_gates[gate][pauli].copy()

    def generate_double_instance(self, prep_basis, meas_basis, noise_repetitions, basis_change_gates=None, transpiled=True):

        GATE_SIZE = 2
        n = self.n
        two_qubit_gates = self.two_qubit_gates
        context_qubits = self.context_qubits
        circ = QuantumCircuit(np.max(self.layer)+1)

        if transpiled:
            gate_dict = self.transpiled_gates
        else: 
            gate_dict = self.raw_gates

        def without_phase(pauli):
            return Pauli((pauli.z, pauli.x))
        
        def prep(pauli_basis):
            if pauli_basis.equiv(Pauli("X")):
                return "H"
            elif pauli_basis.equiv(Pauli("Y")):
                return "HS"
            else:
                return ""

        def meas(pauli_basis, qubit, qc):
            if pauli_basis.equiv(Pauli("X")):
                qc.h(qubit)
            elif pauli_basis.equiv(Pauli("Y")):
                qc.sdg(qubit)
                qc.h(qubit)

        def apply(two_qubit_gate, qc):
            gate,support = two_qubit_gate
            if gate == 'cx':
                qc.cx(*support)
            if gate == 'cz':
                qc.cz(*support)


        #first_twirl = choices(pauli_basis(1, pauli_list = True), k=n)
        first_twirl = Pauli("I"*n)

        for qubit,p,b in zip(self.layer, first_twirl, prep_basis):
            gate_name = prep(b)
            gate_name += p.to_label()
            circ = circ.compose(gate_dict[gate_name], [qubit])

        if noise_repetitions > 0:
            for gate,support in two_qubit_gates:
                apply((gate,support), circ)

        circ.barrier()

        two_qubit_layers = []
        if context_qubits == []:
            context_qubit_layers = [[None]*(noise_repetitions-1)]
        else:
            context_qubit_layers = []
        two_qubit_final_ops = []
        context_qubit_final_ops = []

        for gate,support in two_qubit_gates:

            op = first_twirl[self.layer.index(support[1])].tensor(first_twirl[self.layer.index(support[0])])
            op = self.get_conjugate(gate, op)

            two_qubit_layer = []

            for j in range(noise_repetitions-1):
                #twirl = choice(pauli_basis(GATE_SIZE, pauli_list = True))
                twirl = Pauli("II")
                two_qubit_layer.append(twirl)
                for i,p in enumerate(op):
                    b = basis_change_gates[::-1][self.layer.index(support[i])]
                    if b:
                        op[i] = self.get_conjugate(b,p)
            
                op = without_phase(op.compose(twirl))
                op = self.get_conjugate(gate, op)
            two_qubit_layers.append(two_qubit_layer)
            two_qubit_final_ops.append(op)

        for qubit in context_qubits:
            op = first_twirl[self.layer.index(qubit)]
            context_layer = []
            for i in range(noise_repetitions-1):
                twirl = choice(pauli_basis(1, pauli_list=True))
                op = without_phase(op.compose(twirl))
                context_layer.append(twirl.to_label())
            context_qubit_layers.append(context_layer)
            context_qubit_final_ops.append(op)

        for j,(single_layer, double_layer) in enumerate(zip(list(zip(*context_qubit_layers)), list(zip(*two_qubit_layers)))):
            for single_gate, qubit in zip(single_layer, context_qubits):
                circ = circ.compose(gate_dict[single_gate], [qubit])

            for double_gate, (gate, support) in zip(double_layer, two_qubit_gates):
                for p, qubit in zip(double_gate, support):
                    gate_name = ""
                    b = basis_change_gates[::-1][self.layer.index(qubit)]
                    if b: 
                        gate_name += b
                    gate_name += p.to_label()
                    circ = circ.compose(gate_dict[gate_name], [qubit])
                apply((gate,support), circ)
            
            circ.barrier()

        rostring = "".join(choices(['I','X'], k=n))

        qc = QuantumCircuit(np.max(self.layer)+1)

        for op,(gate,support) in zip(two_qubit_final_ops, two_qubit_gates):
            for p,qubit in zip(op, support):
                qc.append(p, [qubit])
                mapped_index = self.layer.index(qubit)
                s = basis_change_gates[::-1][mapped_index]
                if s:
                    qc = qc.compose(gate_dict[s], [qubit])
                b = meas_basis[mapped_index]
                meas(b, qubit, qc)
                if rostring[::-1][mapped_index] == "X":
                    qc.x(qubit)

        for op,qubit in zip(context_qubit_final_ops, context_qubits):
            mapped_index = self.layer.index(qubit)
            qc.append(op, [qubit])
            b = meas_basis[mapped_index]
            meas(b, qubit, qc)
            gate_name += {"X":"X", "I":""}[rostring[::-1][mapped_index]]
            if rostring[::-1][mapped_index] == "X":
                qc.x(qubit)

        if transpiled:
            qc = transpile(qc, self.backend, optimization_level = 1)

        circ = circ.compose(qc)

        return circ, {"prep_basis":prep_basis, "meas_basis":meas_basis, "length":noise_repetitions, "rostring":rostring}
    
    def get_benchmark_paulis(self):
        n = len(self.adjacency_matrix)
        pauli_list = []
        idPauli = Pauli("I"*n)    
    
        #get all single-weight paulis
        for i in range(n):
            for op in ['X','Y','Z']:
                pauli = idPauli.copy()
                pauli[i] = Pauli(op)
                pauli_list.append(pauli)
                
        #get all weight-two paulis on nieghboring qubits
        for vert1,link in enumerate(self.adjacency_matrix):
            for vert2,val in enumerate(link[:vert1]):
                if val == 1:
                    for pauli1, pauli2 in product(['X','Y','Z'], repeat = 2):
                        pauli = idPauli.copy()
                        pauli[vert1] = Pauli(pauli1)
                        pauli[vert2] = Pauli(pauli2)
                        pauli_list.append(pauli)

        return pauli_list

    def get_pauli_pairs(self):
        GATE_SIZE =2
        benchmark_paulis = self.get_benchmark_paulis()
        conjugate_paulis = []
        for pauli in benchmark_paulis:
            total = Pauli("I"*self.n)

            for qubit in self.context_qubits:
                idx = self.layer.index(qubit)
                total[idx] = pauli[idx]

            for gate,support in self.two_qubit_gates:
                support = tuple(self.layer.index(s) for s in support)
                conjugate = Pauli("I"*GATE_SIZE)
                conjugate[0] = pauli[support[0]]
                conjugate[1] = pauli[support[1]]
                conjugate = self.get_conjugate(gate, conjugate)
                total[support[0]] = conjugate[0]
                total[support[1]] = conjugate[1]
            conjugate_paulis.append(total)

        return list(zip(benchmark_paulis, conjugate_paulis))

    def get_pauli_sets(self):

        pairs = self.get_pauli_pairs()

        def weight(pauli):
            label = pauli.to_label()
            return len(label) - label.count("I")

        easy = [(p1,p2) for p1,p2 in pairs if p1 == p2 or weight(p2) > 2]
        medium = [(p1,p2) for p1,p2 in pairs if weight(p1) == weight(p2) and p1 != p2] 
        hard = [(p1,p2) for p1, p2 in pairs if weight(p1) < weight(p2) and weight(p1) <= 2 and weight(p2) <= 2]
    
        return {"easy":easy,"medium":medium,"hard":hard}

    def generate_measurement_circuits(self):
        bases = self.pauli_strings
        depths = self.depths
        samples = self.samples
        easy = self.pauli_sets['easy']
        medium = self.pauli_sets['medium']
        hard = self.pauli_sets['hard']
        circuits = []
        metadatas = []
        
        #make measurements in all the bases
        for basis, d, s in product(bases, depths, samples):
            circ, data = self.generate_double_instance(basis, basis, d)
        
        circuits.append(circ)
        metadatas.append(data)

        #use single-qubit gates to measure medium set

        #determine how many measurements are needed to complete the model
        

        #make measurements at zero and one depth for these extra operators

Question: do the layers have to be self-adjoint?

In [2783]:
from qiskit.providers.aer import Aer, AerSimulator
backend = Aer.get_backend('qasm_simulator')
from qiskit.providers.fake_provider import FakeVigo
backend = AerSimulator.from_backend(FakeVigo())

Check that all gates are conjugated correctly through layer

In [2784]:
layer = [0,1]
two_qubit_gates = [("cx",(0,1))]
tglp = TwoQubitGateLayerExperiment(layer, backend, two_qubit_gates)
pairs = tglp.get_pauli_pairs()
for p, pdg in pairs:
    qc = QuantumCircuit(5)
    qc.append(p, layer)
    for gate,support in two_qubit_gates:
        if gate == 'cx':
            qc.cx(*support)
        else:
            qc.cz(*support)
    qc.append(pdg,layer)
    qc.measure_all()
    count = sim.run(qc).result().get_counts()
    for key in count.keys():
        if key != "00000":
            print(p,pdg)

In [2785]:
tglp.pauli_sets['medium']

[(Pauli('YX'), Pauli('ZY')),
 (Pauli('YY'), Pauli('ZX')),
 (Pauli('ZX'), Pauli('YY')),
 (Pauli('ZY'), Pauli('YX'))]

In [2794]:
#put in the gates that get back to the original from the pair
circ, metadata = tglp.generate_double_instance(Pauli("YX"), Pauli("YX"), 2, ["HS","Sdg"], transpiled=False)

In [2795]:
circ.draw()

┌───┐┌───┐           ░ ┌─────┐┌───┐           ░ ┌───┐┌─────┐┌───┐ ┌───┐ »
q_0: ┤ H ├┤ I ├───────■───░─┤ Sdg ├┤ I ├───────■───░─┤ I ├┤ Sdg ├┤ H ├─┤ X ├─»
     ├───┤├───┤┌───┐┌─┴─┐ ░ └┬───┬┘├───┤┌───┐┌─┴─┐ ░ ├───┤└┬───┬┘├───┤┌┴───┴┐»
q_1: ┤ H ├┤ S ├┤ I ├┤ X ├─░──┤ H ├─┤ S ├┤ I ├┤ X ├─░─┤ I ├─┤ H ├─┤ S ├┤ Sdg ├»
     └───┘└───┘└───┘└───┘ ░  └───┘ └───┘└───┘└───┘ ░ └───┘ └───┘ └───┘└─────┘»
«               
«q_0: ──────────
«     ┌───┐┌───┐
«q_1: ┤ H ├┤ X ├
«     └───┘└───┘

In [2796]:
circ.measure_all()
sim = Aer.get_backend('qasm_simulator')
sim.run(circ).result().get_counts()

{'00': 511, '11': 513}

In [2793]:
metadata

{'prep_basis': Pauli('ZX'),
 'meas_basis': Pauli('ZX'),
 'length': 2,
 'rostring': 'II'}

In [2231]:
Pauli("XXXYYYZZZ")[slice(4,2, -1)]

Pauli('YY')

In [2232]:
list(range(2,-1,-1))

[2, 1, 0]

In [2233]:
support = (4,3)

In [2234]:
slice(*sorted(support))

slice(3, 4, None)

In [2235]:
tglp.print_gate_conversions().draw()

global phase: 7π/4
       ┌───┐            ░    ┌───┐                            
 q_0: ─┤ I ├────────────░────┤ I ├────────────────────────────
       ├───┤            ░    ├───┤                            
 q_1: ─┤ X ├────────────░────┤ X ├────────────────────────────
       ├───┤            ░  ┌─┴───┴──┐    ┌───┐                
 q_2: ─┤ Y ├────────────░──┤ Rz(-π) ├────┤ X ├────────────────
       ├───┤            ░  ├───────┬┘    ├───┤                
 q_3: ─┤ Z ├────────────░──┤ Rz(π) ├─────┤ I ├────────────────
       ├───┤ ┌───┐      ░ ┌┴───────┴┐    ├───┴┐   ┌─────────┐ 
 q_4: ─┤ H ├─┤ I ├──────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├─
       ├───┤ ├───┤      ░ ├─────────┴┐   ├────┤   ├─────────┤ 
 q_5: ─┤ H ├─┤ X ├──────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├─
       ├───┤ ├───┤      ░ ├──────────┤   ├────┤   ├─────────┴┐
 q_6: ─┤ H ├─┤ Y ├──────░─┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├
       ├───┤ ├───┤      ░ ├─────────┬┘   ├────┤   ├──────────┤
 q_7: ─┤ H ├─┤ Z ├──────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├
       ├───┤ └───┘      ░ ├─────────┤    ├────┤   ├─────────┬┘
 q_8: ─┤ H ├────────────░─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├─
       ├───┤ ┌───┐┌───┐ ░ ├─────────┤    ├────┤   └┬────────┤ 
 q_9: ─┤ H ├─┤ S ├┤ I ├─░─┤ Rz(π/2) ├────┤ √X ├────┤ Rz(-π) ├─
       ├───┤ ├───┤├───┤ ░ ├─────────┴┐   ├────┤    └────────┘ 
q_10: ─┤ H ├─┤ S ├┤ X ├─░─┤ Rz(-π/2) ├───┤ √X ├───────────────
       ├───┤ ├───┤├───┤ ░ ├──────────┤   ├────┤    ┌────────┐ 
q_11: ─┤ H ├─┤ S ├┤ Y ├─░─┤ Rz(-π/2) ├───┤ √X ├────┤ Rz(-π) ├─
       ├───┤ ├───┤├───┤ ░ ├─────────┬┘   ├────┤    └────────┘ 
q_12: ─┤ H ├─┤ S ├┤ Z ├─░─┤ Rz(π/2) ├────┤ √X ├───────────────
       ├───┤ ├───┤└───┘ ░ ├─────────┤    ├────┤    ┌────────┐ 
q_13: ─┤ H ├─┤ S ├──────░─┤ Rz(π/2) ├────┤ √X ├────┤ Rz(-π) ├─
      ┌┴───┴┐├───┤┌───┐ ░ └──┬────┬─┘ ┌──┴────┴─┐  └────────┘ 
q_14: ┤ Sdg ├┤ H ├┤ I ├─░────┤ √X ├───┤ Rz(π/2) ├─────────────
      ├─────┤├───┤├───┤ ░  ┌─┴────┴─┐ └──┬────┬─┘ ┌─────────┐ 
q_15: ┤ Sdg ├┤ H ├┤ X ├─░──┤ Rz(-π) ├────┤ √X ├───┤ Rz(π/2) ├─
      ├─────┤├───┤├───┤ ░  ├────────┤    ├────┤   ├─────────┴┐
q_16: ┤ Sdg ├┤ H ├┤ Y ├─░──┤ Rz(-π) ├────┤ √X ├───┤ Rz(-π/2) ├
      ├─────┤├───┤├───┤ ░  └─┬────┬─┘ ┌──┴────┴──┐└──────────┘
q_17: ┤ Sdg ├┤ H ├┤ Z ├─░────┤ √X ├───┤ Rz(-π/2) ├────────────
      ├─────┤├───┤└───┘ ░    ├────┤   ├─────────┬┘            
q_18: ┤ Sdg ├┤ H ├──────░────┤ √X ├───┤ Rz(π/2) ├─────────────
      └┬───┬┘├───┤      ░ ┌──┴────┴─┐ └─────────┘             
q_19: ─┤ S ├─┤ I ├──────░─┤ Rz(π/2) ├─────────────────────────
       ├───┤ ├───┤      ░ ├─────────┤    ┌───┐                
q_20: ─┤ S ├─┤ X ├──────░─┤ Rz(π/2) ├────┤ X ├────────────────
       ├───┤ ├───┤      ░ └──┬───┬──┘ ┌──┴───┴──┐             
q_21: ─┤ S ├─┤ Y ├──────░────┤ X ├────┤ Rz(π/2) ├─────────────
       ├───┤ ├───┤      ░ ┌──┴───┴───┐└─────────┘             
q_22: ─┤ S ├─┤ Z ├──────░─┤ Rz(-π/2) ├────────────────────────
       ├───┤ └───┘      ░ ├─────────┬┘                        
q_23: ─┤ S ├────────────░─┤ Rz(π/2) ├─────────────────────────
      ┌┴───┴┐┌───┐      ░ ├─────────┴┐                        
q_24: ┤ Sdg ├┤ I ├──────░─┤ Rz(-π/2) ├────────────────────────
      ├─────┤├───┤      ░ ├──────────┤   ┌───┐                
q_25: ┤ Sdg ├┤ X ├──────░─┤ Rz(-π/2) ├───┤ X ├────────────────
      ├─────┤├───┤      ░ └──┬───┬───┘┌──┴───┴───┐            
q_26: ┤ Sdg ├┤ Y ├──────░────┤ X ├────┤ Rz(-π/2) ├────────────
      ├─────┤├───┤      ░ ┌──┴───┴──┐ └──────────┘            
q_27: ┤ Sdg ├┤ Z ├──────░─┤ Rz(π/2) ├─────────────────────────
      ├─────┤└───┘      ░ ├─────────┴┐                        
q_28: ┤ Sdg ├───────────░─┤ Rz(-π/2) ├────────────────────────
      └─────┘           ░ └──────────┘

In [2236]:
tglp.transpiled_gates["HIH"].draw()

KeyError: 'HIH'

In [2261]:
tglp.conjugate_gates['cx'][Pauli("XX")]

Pauli('IX')

In [2262]:
tglp.conjugate_gates['cx'].keys()

dict_keys([Pauli('II'), Pauli('IX'), Pauli('IY'), Pauli('IZ'), Pauli('XI'), Pauli('XX'), Pauli('XY'), Pauli('XZ'), Pauli('YI'), Pauli('YX'), Pauli('YY'), Pauli('YZ'), Pauli('ZI'), Pauli('ZX'), Pauli('ZY'), Pauli('ZZ')])